In [69]:
import xmlrpc.client
from datetime import datetime
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from prettytable import PrettyTable
import math
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Connection parameters
url = 'http://aroerp.com'
db = 'student_project3'
username = 'K.Nilesh-Bandu@liverpool.ac.uk'
password = 'd8e7c7686375b744c5bc1938bd20faf4faab62ca'

# Establish connection
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
uid = common.authenticate(db, username, password, {})

# Create object proxy
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

# Define the fields we want to retrieve for inventory data
inventory_fields = ['name', 'qty_available', 'virtual_available', 'reordering_min_qty', 'reordering_max_qty']

# Search and read product data
inventory_data = models.execute_kw(db, uid, password,
    'product.template', 'search_read',
    [[]], # Domain (empty list means no domain, i.e., fetch all records)
    {'fields': inventory_fields})

# Create a PrettyTable object
table = PrettyTable()

# Set field names
table.field_names = ["Product Name", "Quantity On Hand", "Forecasted Quantity", "Reordering Min Qty", "Reordering Max Qty"]

# Add rows to the table
for product in inventory_data:
    table.add_row([
        product['name'],
        product['qty_available'],
        product['virtual_available'],
        product.get('reordering_min_qty', 'N/A'),
        product.get('reordering_max_qty', 'N/A')
    ])

# Set alignment for each column
table.align["Product Name"] = "l"  # Left align
table.align["Quantity On Hand"] = "r"  # Right align
table.align["Forecasted Quantity"] = "r"  # Right align
table.align["Reordering Min Qty"] = "r"  # Right align
table.align["Reordering Max Qty"] = "r"  # Right align

# Display the table
print(table)

# Convert the product data to a dictionary for easier access
inventory_dict = {product['name']: {
    'qty_available': product['qty_available'],
    'virtual_available': product['virtual_available'],
    'reordering_min_qty': product.get('reordering_min_qty', 'N/A'),
    'reordering_max_qty': product.get('reordering_max_qty', 'N/A')
} for product in inventory_data}

# Fetch sales data
start_date = datetime(2023, 10, 1)
sales_fields = ['product_id', 'date', 'product_uom_qty']

sales_data = models.execute_kw(db, uid, password,
    'sale.report', 'search_read',
    [[('date', '>=', start_date.strftime('%Y-%m-%d'))]], 
    {'fields': sales_fields})

# Process sales data
product_sales_quantity = {}
unique_dates = set()

for sale in sales_data:
    product = sale['product_id'][1]
    try:
        date_obj = datetime.strptime(sale['date'], '%Y-%m-%d %H:%M:%S')
    except ValueError:
        try:
            date_obj = datetime.strptime(sale['date'], '%Y-%m-%d')
        except ValueError:
            print(f"Unexpected date format: {sale['date']}")
            continue
    
    # Set the day to 28th of each month
    date_28th = date_obj.replace(day=28)
    date_str = date_28th.strftime("%d/%m/%Y")
    quantity = sale['product_uom_qty']
    
    unique_dates.add(date_str)
    
    if product not in product_sales_quantity:
        product_sales_quantity[product] = {}
    
    product_sales_quantity[product][date_str] = product_sales_quantity[product].get(date_str, 0) + quantity

dates = sorted(list(unique_dates), key=lambda x: datetime.strptime(x, "%d/%m/%Y"))

# Create df_quantity
df_quantity = pd.DataFrame(product_sales_quantity).T
df_quantity.index.name = 'Product'
df_quantity.columns = pd.to_datetime(df_quantity.columns, format="%d/%m/%Y")

# Function to match product names
def match_product_names(name1, name2, threshold=0.7):
    name1 = name1.lower()
    name2 = name2.lower()
    words1 = set(name1.split())
    words2 = set(name2.split())
    common_words = words1.intersection(words2)
    if len(words1) == 0 or len(words2) == 0:
        return False
    similarity = len(common_words) / max(len(words1), len(words2))
    return similarity >= threshold

# Forecasting models
def simple_moving_average(data, window=3, periods=5):
    ma = data.rolling(window=window).mean()
    return pd.Series([ma.iloc[-1]] * periods, index=pd.date_range(start=data.index[-1], periods=periods+1, freq='M')[1:])

def exponential_smoothing(data, periods=5):
    model = ExponentialSmoothing(data)
    model_fit = model.fit()
    return model_fit.forecast(periods)

def prophet_forecast(data, periods=5):
    df = pd.DataFrame({'ds': data.index, 'y': data.values})
    model = Prophet(weekly_seasonality=False, daily_seasonality=False)
    model.fit(df)
    future = model.make_future_dataframe(periods=periods, freq='M')
    forecast = model.predict(future)
    return forecast.iloc[-periods:]['yhat']

def xgboost_forecast(data, periods=5):
    df = pd.DataFrame({'ds': range(len(data)), 'y': data.values})
    train, test = train_test_split(df, test_size=0.2, shuffle=False)
    model = XGBRegressor(objective='reg:squarederror')
    model.fit(train[['ds']], train['y'])
    future = pd.DataFrame({'ds': range(len(data), len(data) + periods)})
    return pd.Series(model.predict(future), index=pd.date_range(start=data.index[-1], periods=periods+1, freq='M')[1:])

def arima_predict(data, periods=5):
    model = ARIMA(data, order=(1,1,1))
    results = model.fit()
    forecast = results.forecast(steps=periods)
    return forecast

# Function to calculate accuracy
def calculate_accuracy(test_data, forecast):
    mse = mean_squared_error(test_data, forecast)
    accuracy = 100 - np.sqrt(mse)
    return accuracy

# Function to analyze product data
def analyze_product_data():
    # Find the latest date in the sales data
    latest_sale_date = df_quantity.columns.max()
    
    # Calculate the start date for the forecast (next month after the latest sale)
    forecast_start = latest_sale_date + pd.offsets.MonthEnd(1) + pd.offsets.MonthBegin(1)
    
    # Calculate the end date (6 months from the start)
    forecast_end = forecast_start + pd.offsets.MonthEnd(5)
    
    # Generate the forecast months
    forecast_months = pd.date_range(start=forecast_start, end=forecast_end, freq='M')

    product_forecasts = {}
    products_with_forecast = []

    for product in df_quantity.index:
        product_data = df_quantity.loc[product].dropna()

        # Match the product name with inventory data
        best_match = None
        best_match_score = 0

        for inventory_product in inventory_dict.keys():
            score = match_product_names(product, inventory_product)
            if score > best_match_score:
                best_match_score = score
                best_match = inventory_product

        website_forecast = inventory_dict.get(best_match, {}).get('virtual_available') if best_match else None
        quantity_on_hand = inventory_dict.get(best_match, {}).get('qty_available') if best_match else None
        reordering_min_qty = inventory_dict.get(best_match, {}).get('reordering_min_qty') if best_match else None
        reordering_max_qty = inventory_dict.get(best_match, {}).get('reordering_max_qty') if best_match else None

        if len(product_data) < 5:
            product_forecasts[product] = (product_data, None, forecast_months, website_forecast, quantity_on_hand, None, None, None, reordering_min_qty, reordering_max_qty)
        else:
            try:
                # Split data into train and test sets
                train_size = int(len(product_data) * 0.8)
                train_data = product_data[:train_size]
                test_data = product_data[train_size:]

                ma_predictions = simple_moving_average(train_data, periods=len(forecast_months))
                es_predictions = exponential_smoothing(train_data, periods=len(forecast_months))
                prophet_predictions = prophet_forecast(train_data, periods=len(forecast_months))
                xgb_predictions = xgboost_forecast(train_data, periods=len(forecast_months))
                arima_predictions = arima_predict(train_data, periods=len(forecast_months))

                predictions = {
                    'Moving Average': ma_predictions,
                    'Exponential Smoothing': es_predictions,
                    'Prophet': prophet_predictions,
                    'XGBoost': xgb_predictions,
                    'ARIMA': arima_predictions
                }

                accuracies = {method: calculate_accuracy(test_data, forecast[:len(test_data)]) 
                              for method, forecast in predictions.items()}

                best_method = max(accuracies, key=accuracies.get)

                product_forecasts[product] = (product_data, predictions, forecast_months, website_forecast, quantity_on_hand, test_data, accuracies, best_method, reordering_min_qty, reordering_max_qty)
                products_with_forecast.append(product)
            except Exception as e:
                print(f"Error processing product {product}: {str(e)}")
                product_forecasts[product] = (product_data, None, forecast_months, website_forecast, quantity_on_hand, None, None, None, reordering_min_qty, reordering_max_qty)

    return product_forecasts, products_with_forecast



# List of service products to be removed
service_products = [
    "Credit / Discount", "CREDIT", "Delivery charges", "DEL", "DHL Economy Select", "DHLECON",
    "DHL Economy Select (DG Shipment)", "DHLECONDG", "DHL Express Worldwide", "DHLESWW",
    "DHL Express Worldwide (DG Shipment)", "DHLESWWDG", "DOMESTIC UK", "DHLDOM",
    "Down payment", "MISC-COO Certificate of Origin from Chamber of Commerce", "MISC-COO",
    "Third-party delivery charges CLI2060 - Acetal copolymer screw tubing clip up to 14mm diameter", "LC00230",
    "SIE2212 - Endecotts Sieve Brass 200mm x 75um", "LC00235", "TRAVEL", "Travel Costs"
]

if __name__ == '__main__':
    # Analyze the data
    product_forecasts, products_with_forecast = analyze_product_data()

    # Create a list to store the data for the Excel file
    excel_data = []

    for product, data in product_forecasts.items():
        # Skip service products
        if product in service_products:
            continue

        product_data, predictions, forecast_months, website_forecast, quantity_on_hand, test_data, accuracies, best_method, reordering_min_qty, reordering_max_qty = data

        if best_method is not None and quantity_on_hand is not None and reordering_min_qty not in (None, 'N/A'):
            best_forecast = [round_up(value) for value in predictions[best_method]]
            
            # Calculate inventory months left
            months_left, _ = calculate_inventory_months_left(
                round_up(quantity_on_hand), 
                best_forecast, 
                round_up(reordering_min_qty)
            )
            
            # Create a row for each product
            row = {
                "Product Name": product,
                "Quantity on Hand": round_up(quantity_on_hand),
                "Forecasted Quantity by API": round_up(website_forecast),
                "Best Model Used for Prediction": best_method,
                "Reordering Min Quantity": round_up(reordering_min_qty),
                "Inventory Months Left": months_left
            }

            # Add forecasted values for each month
            for date, value in zip(forecast_months, best_forecast):
                month_year = date.strftime("%b/%Y")
                row[f"Forecast {month_year}"] = value

            excel_data.append(row)

    # Create a DataFrame from the collected data
    df_excel = pd.DataFrame(excel_data)

    # Round up all numeric values in the DataFrame
    df_excel = df_excel.applymap(round_up)

    # Generate the Excel file with current timestamp (macOS compatible)
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    excel_filename = f"product_forecasts_{current_time}.xlsx"
    df_excel.to_excel(excel_filename, index=False)

    # Apply conditional formatting
    wb = load_workbook(excel_filename)
    ws = wb.active

    # Find the column index for "Inventory Months Left"
    inventory_months_col = None
    for col in range(1, ws.max_column + 1):
        if ws.cell(row=1, column=col).value == "Inventory Months Left":
            inventory_months_col = col
            break

    if inventory_months_col:
        # Apply red fill to cells with value less than 3
        red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')
        for row in range(2, ws.max_row + 1):  # Start from row 2 to skip header
            cell = ws.cell(row=row, column=inventory_months_col)
            if isinstance(cell.value, (int, float)) and cell.value < 3:
                cell.fill = red_fill

    wb.save(excel_filename)
    print(f"Excel file '{excel_filename}' has been generated with conditional formatting.")



# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='All Products Forecast', children=[
            html.Div([
                dcc.Dropdown(
                    id='product-dropdown',
                    options=[{'label': f'{product_id}', 'value': product_id} for product_id in product_forecasts.keys()],
                    value=list(product_forecasts.keys())[0]
                ),
                dcc.Graph(id='forecast-graph')
            ])
        ]),
        dcc.Tab(label='Products with Completed Predictions', children=[
            html.Div([
                dcc.Dropdown(
                    id='forecast-product-dropdown',
                    options=[{'label': f'{product_id}', 'value': product_id} for product_id in products_with_forecast],
                    value=products_with_forecast[0] if products_with_forecast else None
                ),
                dcc.Graph(id='completed-predictions-graph')
            ])
        ])
    ])
])

@app.callback(
    Output('forecast-graph', 'figure'),
    [Input('product-dropdown', 'value')]
)
def update_graph(selected_product):
    data = product_forecasts.get(selected_product, (None, None, None, None, None, None, None, None, None, None))
    
    if len(data) < 10:
        return px.line(title=f'Product {selected_product} Sales Forecast (Data Incomplete)')
    
    product_data, predictions, forecast_months, website_forecast, quantity_on_hand, test_data, accuracies, best_method, reordering_min_qty, reordering_max_qty = data

    # Create a DataFrame for plotting
    df = pd.DataFrame({'Date': product_data.index, 'Actual': product_data.values})

    # Add predictions to the DataFrame
    if predictions is not None:
        for method, forecast in predictions.items():
            method_with_accuracy = f"{method} ({accuracies[method]:.2f}%)"
            forecast_df = pd.DataFrame({'Date': forecast_months, method_with_accuracy: forecast})
            df = pd.merge(df, forecast_df, on='Date', how='outer')
    
    # Add test data if available and valid
    if test_data is not None and isinstance(test_data, (pd.Series, pd.DataFrame)):
        test_df = pd.DataFrame({'Date': test_data.index, 'Test Data': test_data.values})
        df = pd.merge(df, test_df, on='Date', how='outer')

    # Create the plot
    fig = px.line(df, x='Date', y=df.columns[1:], title=f'Product {selected_product} Sales Forecast')

    # Add website forecast if available
    if website_forecast is not None:
        fig.add_scatter(x=[forecast_months[0]], y=[website_forecast], mode='markers', name='Website Forecast',
                        marker=dict(color='black', size=10, symbol='star'))
        fig.add_annotation(x=forecast_months[0], y=website_forecast,
                           text=f'Website Forecast: {website_forecast}',
                           showarrow=True, arrowhead=7, ax=0, ay=-40)
    
    # Add quantity on hand if available
    if quantity_on_hand is not None:
        fig.add_scatter(x=[forecast_months[0]], y=[quantity_on_hand], mode='markers', name='Quantity On Hand',
                        marker=dict(color='red', size=10, symbol='cross'))
        fig.add_annotation(x=forecast_months[0], y=quantity_on_hand,
                           text=f'Quantity On Hand: {quantity_on_hand}',
                           showarrow=True, arrowhead=7, ax=0, ay=-60)  # Adjusted y-offset

    # Add reordering min and max quantities to the plot
    if reordering_min_qty is not None and reordering_min_qty != 'N/A':
        fig.add_hline(y=reordering_min_qty, line_dash="dash", line_color="orange", annotation_text="Reordering Min Qty")
    if reordering_max_qty is not None and reordering_max_qty != 'N/A':
        fig.add_hline(y=reordering_max_qty, line_dash="dash", line_color="red", annotation_text="Reordering Max Qty")

    # Stock Depletion Line based on Best Forecast Method
    if quantity_on_hand is not None and best_method is not None:
        depletion_values = []
        remaining_stock = quantity_on_hand

        for date, forecast in zip(forecast_months, predictions[best_method]):
            remaining_stock -= forecast
            depletion_values.append(remaining_stock)
            if remaining_stock <= 0:
                break

    # Stock Depletion Line based on Best Forecast Method
    if quantity_on_hand is not None and best_method is not None:
        depletion_values = []
        remaining_stock = quantity_on_hand

        for date, forecast in zip(forecast_months, predictions[best_method]):
            remaining_stock -= forecast
            depletion_values.append(remaining_stock)
            if remaining_stock <= 0:
                break

        # Add Stock Depletion Line to the plot
        depletion_dates = forecast_months[:len(depletion_values)]
        fig.add_scatter(x=depletion_dates, y=depletion_values, mode='lines', name='Stock Depletion',
                        line=dict(color='purple', dash='dash'))
        
    # Show the best forecast method and its percentage at the bottom
    if accuracies:
        best_accuracy = accuracies[best_method]
        fig.add_annotation(
            x=0.5, y=-0.15, xref='paper', yref='paper',
            text=f'Best Forecast Method: {best_method} ({best_accuracy:.2f}%)',
            showarrow=False, font=dict(size=14, color='green'),
            align='center', yanchor='top'
        )

    # Update layout
    fig.update_layout(legend_title_text='Forecast Method', xaxis_title='Date', yaxis_title='Sales',
                      xaxis=dict(tickmode='linear', dtick='M1'))  # Show all months

    return fig

@app.callback(
    Output('completed-predictions-graph', 'figure'),
    [Input('forecast-product-dropdown', 'value')]
)
def update_completed_predictions_graph(selected_product):
    return update_graph(selected_product)

if __name__ == '__main__':
    app.run_server(debug=True)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+---------------------+--------------------+--------------------+
| Product Name                                                                                                                                                                                                                        |   Quantity On Hand | Forecasted Quantity | Reordering Min Qty | Reordering Max Qty |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+---------------------+--------------------+--------------------+
| 00T0796 - TV7000DC CALIBRAT.BATH 230V/50-60HZ  

/var/folders/mf/p54spbqs087cz2fkjv7xjm7h0000gn/T/ipykernel_5084/970093448.py:181: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/var/folders/mf/p54spbqs087cz2fkjv7xjm7h0000gn/T/ipykernel_5084/970093448.py:134: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



Excel file 'product_forecasts_20240930_191450.xlsx' has been generated with conditional formatting.
